In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

# The Objective of the exercise:
## To practice data cleaning and data manipulation on a data set
to gain insights on the data set
This is a generated data for data cleaning exercise, which contains data on an online ecommerce store. 

Finding:
the sales peformance, patterns and insights of this ecommerce dataset over the past 5 years


# Step 1. Importing the Online Sales Dataset for a deeper observation of the dataset

In [2]:
#loading the dataset
df = pd.read_csv('online_sales_dataset.csv')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.470000,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.190000,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00,29.11,50472.0,Germany,0.350000,Bank Transfer,23.03,Electronics,Online,Returned,UPS,Berlin,High
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00,76.68,96586.0,Netherlands,0.140000,paypall,11.08,Accessories,Online,Not Returned,Royal Mail,Rome,Low
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00,-68.11,NaN,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49777,354083,SKU_1562,Blue Pen,25,2025-09-05 01:00,70.92,51445.0,Spain,0.200000,Credit Card,8.96,Electronics,Online,Returned,UPS,Berlin,Medium
49778,296698,SKU_1930,USB Cable,7,2025-09-05 02:00,51.74,28879.0,United States,0.230000,Bank Transfer,23.55,Electronics,Online,Not Returned,FedEx,Amsterdam,Low
49779,177622,SKU_1766,Office Chair,43,2025-09-05 03:00,85.25,21825.0,Portugal,0.200000,Bank Transfer,16.26,Furniture,In-store,Not Returned,FedEx,London,High
49780,701213,SKU_1602,Notebook,48,2025-09-05 04:00,39.64,43199.0,United Kingdom,0.310000,paypall,28.56,Apparel,Online,Not Returned,Royal Mail,London,Medium


### 1.1 Performing EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49782 entries, 0 to 49781
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   InvoiceNo          49782 non-null  int64  
 1   StockCode          49782 non-null  object 
 2   Description        49782 non-null  object 
 3   Quantity           49782 non-null  int64  
 4   InvoiceDate        49782 non-null  object 
 5   UnitPrice          49782 non-null  float64
 6   CustomerID         44804 non-null  float64
 7   Country            49782 non-null  object 
 8   Discount           49782 non-null  float64
 9   PaymentMethod      49782 non-null  object 
 10  ShippingCost       47293 non-null  float64
 11  Category           49782 non-null  object 
 12  SalesChannel       49782 non-null  object 
 13  ReturnStatus       49782 non-null  object 
 14  ShipmentProvider   49782 non-null  object 
 15  WarehouseLocation  46297 non-null  object 
 16  OrderPriority      497

In [4]:
df.describe()

,InvoiceNo,Quantity,UnitPrice,CustomerID,Discount,ShippingCost
count,49782.000000,49782.000000,49782.000000,44804.000000,49782.000000,47293.000000
mean,550681.239946,22.372343,47.537862,55032.871775,0.275748,17.494529
std,260703.009944,17.917774,33.479510,25913.660157,0.230077,7.220557
min,100005.000000,-50.000000,-99.980000,10001.000000,0.000000,5.000000
25%,324543.000000,11.000000,23.592500,32750.750000,0.130000,11.220000
50%,552244.000000,23.000000,48.920000,55165.000000,0.260000,17.500000
75%,776364.000000,37.000000,74.610000,77306.250000,0.380000,23.720000
max,999997.000000,49.000000,100.000000,99998.000000,1.999764,30.000000


In [5]:
#InvoiceDate is object dtype. To change to datetime64 format
#CustomerID is float64. last digit .0 is redundant. to remove and change to int64 format.
#Discount can be rounded to 2 decimal places. Negative values has also been observed in about 10% of the records in Discount feature.
#New Columns can be added for deeper analyis. sales_amt = Quantity x UnitPrice | sales_amt_discounted = Quantity x (UnitPrice x Discount)
#typo error in PaymentMethod. (paypall instead of PayPal)

In [6]:
df.isna().sum()

InvoiceNo               0
StockCode               0
Description             0
Quantity                0
InvoiceDate             0
UnitPrice               0
CustomerID           4978
Country                 0
Discount                0
PaymentMethod           0
ShippingCost         2489
Category                0
SalesChannel            0
ReturnStatus            0
ShipmentProvider        0
WarehouseLocation    3485
OrderPriority           0
dtype: int64

In [7]:
#CustomerID has null values. Replace with '0'
#ShippingCost has null values. Replace with median value of ShippingCost
#WarehouseLocation has null values. Replace with 'Unknown'

# Step 2. Begin Data Cleansing and Transformation

### 2.1 Changing InvoiceDate dtype to  datetime64 format

In [8]:
#InvoiceDate is object dtype. To change to datetime64 format
print("Before Conversion:", df['InvoiceDate'].dtype)

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

Before Conversion: object


In [9]:
print("After Conversion:", df['InvoiceDate'].dtype)

After Conversion: datetime64[ns]


### 2.2 Filling in null values and changing CustomerID to Integer format

In [10]:
#CustomerID is float64. last digit .0 is redundant. to remove and change to int format.

#handling the null values first in CustomerID.
df['CustomerID'].fillna(0, inplace=True)


In [11]:
#Converting CustomerID to int with astype function.
df['CustomerID'] = df['CustomerID'].round().astype(int)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49782 entries, 0 to 49781
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   InvoiceNo          49782 non-null  int64         
 1   StockCode          49782 non-null  object        
 2   Description        49782 non-null  object        
 3   Quantity           49782 non-null  int64         
 4   InvoiceDate        49782 non-null  datetime64[ns]
 5   UnitPrice          49782 non-null  float64       
 6   CustomerID         49782 non-null  int32         
 7   Country            49782 non-null  object        
 8   Discount           49782 non-null  float64       
 9   PaymentMethod      49782 non-null  object        
 10  ShippingCost       47293 non-null  float64       
 11  Category           49782 non-null  object        
 12  SalesChannel       49782 non-null  object        
 13  ReturnStatus       49782 non-null  object        
 14  Shipme

### 2.3 Rounding off Discount to 2 decimal places

In [13]:
#Discount can be rounded to 2 decimal places.
df["Discount"] = df["Discount"].round(2)
display(df["Discount"])

0        0.47
1        0.19
2        0.35
3        0.14
4        1.50
         ... 
49777    0.20
49778    0.23
49779    0.20
49780    0.31
49781    0.10
Name: Discount, Length: 49782, dtype: float64

### 2.4 Replacing null values in WarehouseLocation

In [14]:
#WarehouseLocation has null values. Replace with 'Unknown'
df['WarehouseLocation'].fillna('Unknown', inplace=True)
display(df["WarehouseLocation"])

0           London
1             Rome
2           Berlin
3             Rome
4          Unknown
           ...    
49777       Berlin
49778    Amsterdam
49779       London
49780       London
49781         Rome
Name: WarehouseLocation, Length: 49782, dtype: object

### 2.5 Replacing null values in ShippingCost

In [15]:
#ShippingCost has null values. Replace with median value of ShippingCost
df.describe()

,InvoiceNo,Quantity,InvoiceDate,UnitPrice,CustomerID,Discount,ShippingCost
count,49782.000000,49782.000000,49782,49782.000000,49782.000000,49782.000000,47293.000000
mean,550681.239946,22.372343,2022-11-03 02:30:00,47.537862,49529.805693,0.275750,17.494529
min,100005.000000,-50.000000,2020-01-01 00:00:00,-99.980000,0.000000,0.000000,5.000000
25%,324543.000000,11.000000,2021-06-02 13:15:00,23.592500,25227.250000,0.130000,11.220000
50%,552244.000000,23.000000,2022-11-03 02:30:00,48.920000,50025.500000,0.260000,17.500000
75%,776364.000000,37.000000,2024-04-04 15:45:00,74.610000,74794.500000,0.380000,23.720000
max,999997.000000,49.000000,2025-09-05 05:00:00,100.000000,99998.000000,2.000000,30.000000
std,260703.009944,17.917774,NaN,33.479510,29613.149485,0.230086,7.220557


In [16]:
df['ShippingCost'].median()

17.5

In [17]:
df['ShippingCost'].fillna(df['ShippingCost'].median(), inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49782 entries, 0 to 49781
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   InvoiceNo          49782 non-null  int64         
 1   StockCode          49782 non-null  object        
 2   Description        49782 non-null  object        
 3   Quantity           49782 non-null  int64         
 4   InvoiceDate        49782 non-null  datetime64[ns]
 5   UnitPrice          49782 non-null  float64       
 6   CustomerID         49782 non-null  int32         
 7   Country            49782 non-null  object        
 8   Discount           49782 non-null  float64       
 9   PaymentMethod      49782 non-null  object        
 10  ShippingCost       49782 non-null  float64       
 11  Category           49782 non-null  object        
 12  SalesChannel       49782 non-null  object        
 13  ReturnStatus       49782 non-null  object        
 14  Shipme

### 2.6 Correcting a spelling error in PaymentMethod. (PayPal misspelled as paypall)

In [19]:
# Correct the typo error in the 'payment_method' column 
df['PaymentMethod'] = df['PaymentMethod'].str.replace('paypall', 'PayPal')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00:00,1.71,37039,Australia,0.47,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00:00,41.25,19144,Spain,0.19,PayPal,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00:00,29.11,50472,Germany,0.35,Bank Transfer,23.03,Electronics,Online,Returned,UPS,Berlin,High
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00:00,76.68,96586,Netherlands,0.14,PayPal,11.08,Accessories,Online,Not Returned,Royal Mail,Rome,Low
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00:00,-68.11,0,United Kingdom,1.50,Bank Transfer,17.50,Electronics,In-store,Not Returned,FedEx,Unknown,Medium


### 2.7 Handling Abnormal Discount Values in Discount Feature.
There are some records showing abnormal discount levels in the dataset. Discount levels of over 1 doesn't make sense as company will be selling
As the number of affected values is small and no determinant to ascertain if it was a data-entry erroneous input, we have decided to cap the discount level at 1 if the value exceeds 1.

In [20]:
#determining total number of affected records with discount levels above 1.
discount_over_100 = df[df['Discount'] > 1]

# Calculate frequency
count_over_100 = len(discount_over_100)
total_records = len(df)
percentage_over_100 = (count_over_100 / total_records) * 100

# Display frequency and percentage
print(f"Number of records with discount > 100%: {count_over_100}") #less than 2% of the total records affected 992/49,782

Number of records with discount > 100%: 992


In [21]:
#Capping the discount level values to only 1 if value exceeds 1

df['Discount'] = df['Discount'].apply(lambda x: min(x, 1))
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00:00,1.71,37039,Australia,0.47,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00:00,41.25,19144,Spain,0.19,PayPal,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00:00,29.11,50472,Germany,0.35,Bank Transfer,23.03,Electronics,Online,Returned,UPS,Berlin,High
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00:00,76.68,96586,Netherlands,0.14,PayPal,11.08,Accessories,Online,Not Returned,Royal Mail,Rome,Low
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00:00,-68.11,0,United Kingdom,1.00,Bank Transfer,17.50,Electronics,In-store,Not Returned,FedEx,Unknown,Medium


### 2.8 Handling Negative Values in Quantity & UnitPrice Features
Some of the values in Quantity & UnitPrices are recorded in negative values. This is likely an error as products cannot be sold in a negative quantity and price.

In [22]:
#To correct this, we need to change all the negative values in the Quantity and UnitPrice features to positive.
#All negative values will transform to absolute values.
df['Quantity'] = df['Quantity'].abs()
df['UnitPrice'] = df['UnitPrice'].abs()


In [23]:
df.sample(30)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
40637,451416,SKU_1258,Wireless Mouse,46,2024-08-20 05:00:00,18.78,85516,Portugal,0.33,Bank Transfer,24.25,Apparel,Online,Not Returned,FedEx,Paris,Medium
24172,590073,SKU_1986,Desk Lamp,29,2022-10-04 04:00:00,76.39,48134,Germany,0.13,Credit Card,13.00,Stationery,In-store,Not Returned,UPS,Berlin,High
3789,891507,SKU_1349,Office Chair,5,2020-06-06 21:00:00,46.46,85933,Spain,0.38,Credit Card,20.61,Accessories,In-store,Not Returned,FedEx,Rome,High
26801,809609,SKU_1580,White Mug,1,2023-01-21 17:00:00,34.23,68345,Netherlands,0.29,Bank Transfer,23.21,Electronics,Online,Not Returned,FedEx,Amsterdam,Low
32835,137455,SKU_1775,Headphones,7,2023-09-30 03:00:00,7.33,92661,Italy,0.22,Credit Card,28.18,Furniture,In-store,Not Returned,Royal Mail,Rome,Low
12139,952224,SKU_1135,USB Cable,8,2021-05-20 19:00:00,47.74,57569,Netherlands,0.06,Bank Transfer,8.80,Stationery,In-store,Not Returned,FedEx,Rome,High
2787,614003,SKU_1619,Notebook,36,2020-04-26 03:00:00,61.43,38467,United Kingdom,0.22,Credit Card,22.42,Furniture,Online,Not Returned,FedEx,Berlin,Low
26563,517935,SKU_1073,Desk Lamp,28,2023-01-11 19:00:00,41.37,96356,United States,0.02,PayPal,27.53,Accessories,In-store,Not Returned,FedEx,Amsterdam,Medium
40774,343432,SKU_1251,Backpack,44,2024-08-25 22:00:00,27.78,66078,Netherlands,0.45,Credit Card,21.65,Apparel,Online,Not Returned,Royal Mail,Berlin,Medium
334,480002,SKU_1797,USB Cable,1,2020-01-14 22:00:00,89.37,33323,Norway,0.13,PayPal,8.00,Stationery,Online,Not Returned,FedEx,Berlin,High


### 2.9 Creating new features Sales and Sales_Discounted + Year, Quarter, Month

In [24]:
# Adding Sales and Sales_Discounted Columns from calculations of Quantity, UnitPrice and Discount 
df['Sales'] = df['Quantity'] * df['UnitPrice']
df['Sales_Discounted'] = (df['Quantity'] * df['UnitPrice']) * (1 - df['Discount'])

# Adding Year, Quarter and Month Columns from InvoiceDate Column
df['Year'] = df['InvoiceDate'].dt.year 
df['Quarter'] = 'Q' + df['InvoiceDate'].dt.quarter.astype(str)
df['MonthNum'] = df['InvoiceDate'].dt.month
df['Month'] = df['InvoiceDate'].dt.strftime('%b')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49782 entries, 0 to 49781
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   InvoiceNo          49782 non-null  int64         
 1   StockCode          49782 non-null  object        
 2   Description        49782 non-null  object        
 3   Quantity           49782 non-null  int64         
 4   InvoiceDate        49782 non-null  datetime64[ns]
 5   UnitPrice          49782 non-null  float64       
 6   CustomerID         49782 non-null  int32         
 7   Country            49782 non-null  object        
 8   Discount           49782 non-null  float64       
 9   PaymentMethod      49782 non-null  object        
 10  ShippingCost       49782 non-null  float64       
 11  Category           49782 non-null  object        
 12  SalesChannel       49782 non-null  object        
 13  ReturnStatus       49782 non-null  object        
 14  Shipme

### 2.10 Sales in Year 2025 and 2024.
As this report is compiled in 2024, it is not likely that we can generate a sales report that has records in 2025 and 2024.

In [25]:
#To handle this Year error, we will reverse the report timeframe to 2 years backwards. This will also ensure all records are in tact and analyzed.
df['Year'] = df['Year'] - 2

In [26]:
df['Year'].tail()

49777    2023
49778    2023
49779    2023
49780    2023
49781    2023
Name: Year, dtype: int32

In [27]:
df['InvoiceNo'].nunique()

48494

# Online Sales Dataset Cleaning & Transformation completed and exporting to Power BI for further data analysis.

In [28]:
df.to_csv('cleandata.csv')